# D-Hydro De Dellen V1

## Introductie

Met dit notebook bouw je een D-hydromodel voor Stroomgebied De Dellen in het beheergebied van waterschap Hunze en Aa's.

Zorg dat je beschikt over een werkende Python installatie en <a href="https://github.com/openearth/delft3dfmpy#installation">D-HYDAMO omgeving</a>. Het resulterende model moet geimporteerd kunnen worden in D-HYDRO versie 0.9.7.52006 of hoger.

De verwijzing naar bestanden is geschreven in het Nederlands. De rest van de code is geschreven in het Engels. Elk code-blok is voorzien van uitleg boven het desbetreffende blok, geschreven in het Nederlands.

Download de bestanden van <a href="https://www.dropbox.com/s/xzl38m1xyl9dqpc/beheerregister.zip?dl=0">Dropbox</a> en zet deze in de folder .\beheerregister
    
Dependencies: 
- voor het uitlezen van XLSX-bestanden (boundaries) is openpyxl nodig: conda install openpyxl dit moet uitgevoerd worden binnen de delft3dfmpy-environmen.

## Controle bestanden

Alle bestanden die gebruikt worden in deze tutorial staan in het code-blok hieronder. Wanneer je dit codeblok uitvoert wordt de aanwezigheid van deze bestanden gecontroleerd.



In [2]:
from pathlib import Path

data_path = Path(r".\data").absolute().resolve()
excelbestanden = data_path.joinpath("xlsx")

beheerregister = Path(r".\beheerregister").absolute().resolve()

modelbestanden = {"randvoorwaarden":"randvoorwaarden.xlsx"}

invoerbestanden = {"modelgebied": "DeDellen_gebiedsgrens.shp",
                   "branches": "Hoofdwatergang_Dellen_singlepart.shp",
                   "profielpunten": "profielpunten_Dellen.shp",
                   "bruggen": "Brug.shp",
                   "duikers": "Duiker_Dellen.shp",
                   "sifons": "Syphon.shp",
                   "stuwen": "Stuw.shp",
                   "inlaten": "Inlaat.shp",
                   "gemalen": "Gemaal.shp",
                   "peilgebieden": "Peilgebied_Dellen.shp"}

vorm_mapping = dict(rond=1,
                    driehoekig=1,
                    rechthoekig=3,
                    eivormig=1,
                    ellips=1,
                    Paraboolvormig=1,
                    trapeziumvormig=1,
                    heul=1,
                    muil=3,
                    langwerpig=1,
                    scherp=1,
                    onbekend=99,
                    overig=99)

ruwheid_mapping = {"A1": 0.03,
                   "A2": 0.03,
                   "A2 Boot": 0.03,
                   "A3": 0.03,
                   "B1": 0.02,
                   "B2": 0.022,
                   "C1": 0.02,
                   "C2b": 0.022,
                   "DERDEN": 0.03,
                   "GEEN": 0.07}

for key, item in invoerbestanden.items():
    if not beheerregister.joinpath(item).exists():
        print(f"bestand voor {key} bestaat niet: {beheerregister.joinpath(item)}")
        
for key, item in modelbestanden.items():
    if not excelbestanden.joinpath(item).exists():
        print(f"bestand voor {key} bestaat niet: {excelbestanden.joinpath(item)}")
        
print("Paden succesvol ingesteld.")

Paden succesvol ingesteld.


## Inlezen beheerregister in HyDAMO

### Aanmaken HyDAMO object

Alle benodigde modules worden geimporteerd en het HYDAMO object wordt aangemaakt bij het uitvoeren van onderstaand code-blok

In [2]:
from delft3dfmpy import DFlowFMModel, HyDAMO, Rectangular, DFlowFMWriter
from delft3dfmpy import DFlowRRModel, DFlowRRWriter
from delft3dfmpy.datamodels.common import ExtendedGeoDataFrame
import hydrotools
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString

hydamo = HyDAMO(extent_file=str(beheerregister.joinpath(invoerbestanden["modelgebied"])))

### Toevoegen waterlopen

De waterlopen (branches) worden toegevoegd met volgend code-blok:
* de branches worden toegekend aan het hydamo-object
* een HyDAMO ruwheidscode (4 = manning) wordt toegekend
* de eindpunten van de branches worden gesnapped binnen een tolerantie van 1m. De coordinaten worden ook afgerond op 1m.

In [3]:
# gdf = gpd.read_file(beheerregister.joinpath(invoerbestanden["branches"]))
# gdf.columns = gdf.columns.str.lower()
# print(gdf.columns)

gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["branches"]),
                           hydamo_attribute="branches",
                           index_col="ovkident",
                           keep_columns=["code",
                                         "bodembreedte",
                                         "bodemhoogte",
                                         "bovenbreedte",
                                         "taludhellinglinkerzijde",
                                         "avvtalur",
                                         "taludhellingrechterzijde",
                                         "werkcode",
                                         "objectid"],
                           column_mapping={"ovkident": "code",
                                           "avvboddr": "bodembreedte",
                                           "avvhobos": "bodemhoogte",
                                           "iws_bovenb": "bovenbreedte",
                                           "avvtalul": "taludhellinglinkerzijde",
                                           "avvtalur": "taludhellingrechterzijde"}
                           )

gdf.loc[:, "ruwheidstypecode"] = 2
gdf.loc[:, "ruwheidswaarde"] = gdf.apply((lambda x: ruwheid_mapping[x["werkcode"]]), axis=1)

#print(beheerregister.joinpath(invoerbestanden["branches"]))
#print("kolommen in geodataframe: " + gdf.columns)

#hydamo.branches = hydrotools.snap_ends(hydamo.branches, tolerance=1, digits=1)

# verplaatsen eindpunten
#move_lines_gdf = gpd.read_file(data_path.joinpath("shp","verplaats_eind_nodes.shp"))
#gdf = hydrotools.move_end_nodes(gdf, move_lines_gdf, threshold=1)

hydamo.branches.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)

#hydamo.branches = hydrotools.snap_ends(hydamo.branches, tolerance=1, digits=1)

#hydamo.branches = hydamo.branches.loc[~hydamo.branches.index.isin(["OAF016902",
 #                                                                  "OAF016901"])]

### Aanmaken laterale instroompunten RR

In [ ]:
# Laterals (imported from shapefile)
hydamo.laterals.read_shp(os.path.join(data_path,'data/shp/1DLateralInflowPoints.shp'),
                        column_mapping={'OBJECTID': 'code',
                                        'OBJECTID': 'name',
                                        'X':'X',
                                        'Y':'Y'})
hydamo.laterals.snap_to_branch(hydamo.branches, snap_method='overal', maxdist=  500)
hydamo.laterals.dropna(axis=0, inplace=True, subset=['branch_offset'])


### Aanmaken principeprofielen
Als terugvaloptie voor branches zonder profielen schrijven we principeprofielen weg die we later in deze notebook gebruiken

In [4]:
principe_profielen_df = hydrotools.get_trapeziums(gdf,
                                                  "code",
                                                  "bodembreedte",
                                                  "bodemhoogte",
                                                  "bovenbreedte",
                                                  "taludhellinglinkerzijde",
                                                  "taludhellingrechterzijde",
                                                  "ruwheidstypecode",
                                                  "ruwheidswaarde")

principe_profielen_df.to_csv(excelbestanden.joinpath("principe_profielen.csv"))

### Toevoegen yz-profielen
De yz-profielen worden ingeladen:
* De profielpunten worden geopend
* de punten worden geordend en geconverteerd naar polylinen met een xyz coordinaten
* toekennen ruwheid (manning = 35)
* lijnen langer dan 500m worden weggegooid, omdat er soms kademuren PRO_TYPE = PRO hebben gekregen
* lijnen worden toegekend aan het HyDAMO object
* lijnen die niet snappen met de branches, worden weggegooid

ToDo:
* verbeteren toekenning ruwheid

In [5]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["profielpunten"]),
                           "crosssections",
                           attribute_filter={"osmomsch": ["Z1"]},  #we nemen alleen profielen van het type Z1 (vaste bodem) 
                           column_mapping={"CODE": "code",
                                           "IWS_VOLGNR": "order",
                                           "OSMOMSCH": "category",
                                           "iws_hoogte": "z"},
                           z_coord=True
                           )
print(gdf["code"])
grouper = gdf.groupby("code")
profiles = dict()

for code, prof_gdf in grouper:
    if len(prof_gdf) > 1: 
        prof_gdf = prof_gdf.sort_values("order")  #Staan ook punten met zelfde order
        first_point = prof_gdf.iloc[0]["geometry"]
        cum_dist = -1
        line = []
        for idx, (_, row) in enumerate(prof_gdf.iterrows()):
            geom = row["geometry"]
            distance = geom.distance(first_point)
            if distance > cum_dist:
                cum_dist = distance
                line += [(geom.x, geom.y, row["z"])]

        profiles[code] = [code, LineString(line)]
    else:
        print(f"Waarschuwing: {code} bevat slechts één punt")
profiles_gdf = gpd.GeoDataFrame.from_dict(profiles,
                                          orient="index",
                                          columns=["code",
                                                   "geometry"]
                                          )

profiles_gdf["ruwheidstypecode"] = 4
profiles_gdf["ruwheidswaarde"] = 35
profiles_gdf["codegerelateerdobject"] = None

profiles_gdf = profiles_gdf.loc[profiles_gdf["geometry"].length < 500]

hydamo.crosssections.set_data(profiles_gdf,
                              index_col="code",
                              check_columns=True,
                              check_geotype=True)

hydamo.crosssections.snap_to_branch(hydamo.branches, snap_method="intersecting")
hydamo.crosssections.dropna(axis=0, inplace=True, subset=["branch_offset"])

0          20030408-7
5          20030404-5
7          20030312-7
8          20030310-9
9          20030211-3
            ...      
3980    DP20151028-11
3982    DP20151028-15
3985    DP20151028-12
3986    DP20151028-11
3987    DP20151028-11
Name: code, Length: 2371, dtype: object
Waarschuwing: 20060329-1372 bevat slechts één punt
Waarschuwing: 20060329-1373 bevat slechts één punt
Waarschuwing: DP20130329-36 bevat slechts één punt
Waarschuwing: DP20140224-06 bevat slechts één punt
Waarschuwing: DP20151026-21 bevat slechts één punt
Waarschuwing: LT0196180904113302 bevat slechts één punt


### Toevoegen geparameteriseerde profielen

Per brug moet er een profiel worden toegevoegd

In [6]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["bruggen"]),
                           "parametrised_profiles",
                           column_mapping={
                                           "KBRIDENT":"code",
                                           "KBRBHBO": "bodemhoogtebovenstrooms",
                                           "KBRBHBE": "bodemhoogtebenedenstrooms",
                                           "KBRBREED": "bodembreedte",
                                           "KBRHOBO": "hoogteinsteeklinkerzijde",
                                           })

grouper = gdf.groupby("code")
data = {"code": [code for code, frame in grouper],
        "bodemhoogtebovenstrooms": [frame["bodemhoogtebovenstrooms"].values[0]
                                    for code, frame in grouper],
        "bodemhoogtebenedenstrooms": [frame["bodemhoogtebenedenstrooms"].values[0]
                                      for code, frame in grouper],
        "bodembreedte": [frame["bodembreedte"].values[0]
                         for code, frame in grouper],
        "hoogteinsteeklinkerzijde": [frame["hoogteinsteeklinkerzijde"].values[0]
                                     for code, frame in grouper],
        "geometry": [frame["geometry"].values[0] for code, frame in grouper]
        }

gdf = gpd.GeoDataFrame(data)
gdf["geometry"] = gdf.apply((lambda x: LineString([[x["geometry"].x,
                                                    x["geometry"].y],
                                                  [x["geometry"].x+1,
                                                   x["geometry"].y+1]])),
                            axis=1)

gdf["codegerelateerdobject"] = gdf["code"].copy()
gdf["code"] = [f"PRO_{code}" for code in gdf["code"]]
gdf["hoogteinsteekrechterzijde"] = gdf["hoogteinsteeklinkerzijde"]
gdf["ruwheidswaarde"] = 15
gdf["ruwheidstypecode"] = 4
gdf["taludhellinglinkerzijde"] = 1
gdf["taludhellingrechterzijde"] = 1

#vervang nan-waarde voor bodembreedte door  defaultwaarde 1
gdf.loc[gdf["bodembreedte"].isna(), "bodembreedte"] = 1

hydamo.parametrised_profiles.set_data(gdf,
                                      index_col="code",
                                      check_columns=True,
                                      check_geotype=False)

### Toevoegen bruggen
Per brug moet er een profiel worden toegevoegd

In [7]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["bruggen"]),
                           "bridges",
                           column_mapping={
                                           "KBRIDENT":"code",
                                           "KBRLENGT": "lengte",
                                           "KBRHOBO": "hoogteonderzijde"})

grouper = gdf.groupby("code")
data = {"code": [code for code, frame in grouper],
        "hoogteonderzijde": [frame["hoogteonderzijde"].values[0] for code,
                             frame in grouper],
        "lengte": [frame["lengte"].values[0] for code, frame in grouper],
        "geometry": [frame["geometry"].values[0] for code, frame in grouper]
        }
gdf = gpd.GeoDataFrame(data)

gdf["hoogtebovenzijde"] = gdf["hoogteonderzijde"] + 1

gdf["dwarsprofielcode"] = gdf["code"]
gdf["intreeverlies"] = 0.7
gdf["uittreeverlies"] = 0.7
gdf["ruwheidstypecode"] = 4
gdf["ruwheidswaarde"] = 70

hydamo.bridges.set_data(gdf, index_col="code", check_columns=True, check_geotype=False)
hydamo.bridges.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.bridges.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen duikers en siphons
duikers en sifons worden als cuverts aan het HyDAMO object toegekend

ToDo
* vorm moet naar HyDAMO codering worden omgeschreven

In [8]:
culverts_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["duikers"]),
                                    "culverts",
                                    column_mapping={
                                                    "KDUIDENT": "code",
                                                    "KDULENGT": "lengte",
                                                    "KDUHGA1": "hoogteopening",
                                                    "KDUBREED": "breedteopening",
                                                    "KDUBOKBE": "hoogtebinnenonderkantbenedenstrooms",
                                                    "KDUBOKBO": "hoogtebinnenonderkantbovenstrooms",
                                                    "KDUVORM": "vormcode"})

siphons_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["sifons"]),
                                   "culverts",
                                   column_mapping={
                                                   "KSYIDENT": "code",
                                                   "IWS_LENGTE": "lengte",
                                                   "KSYHGA1": "hoogteopening",
                                                   "KSYBREED": "breedteopening",
                                                   "IWS_HBOKBE":"hoogtebinnenonderkantbenedenstrooms",
                                                   "IWS_HBOKBO":"hoogtebinnenonderkantbovenstrooms",
                                                   "KSYVORM":"vormcode"})

gdf = gpd.GeoDataFrame(pd.concat([culverts_gdf,siphons_gdf], ignore_index=True))

gdf.loc[gdf['vormcode'].isnull(), 'vormcode'] = 'onbekend'
gdf.loc[:, 'vormcode'] = gdf.apply((lambda x: vorm_mapping[x['vormcode'].lower()]), axis=1)

gdf["intreeverlies"] = 0.7
gdf["uittreeverlies"] = 0.7
gdf["ruwheidswaarde"] = 70
gdf["ruwheidstypecode"] = 4

hydamo.culverts.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.culverts.snap_to_branch(hydamo.branches, snap_method="ends", maxdist=5)
hydamo.culverts.dropna(axis=0, inplace=True, subset=["branch_offset"])

hydamo.culverts.loc[hydamo.culverts["hoogteopening"].isna(), "hoogteopening"] = 0.5
hydamo.culverts.loc[hydamo.culverts["breedteopening"].isna(), "breedteopening"] = 0.5
hydamo.culverts.loc[hydamo.culverts["lengte"].isna(), "lengte"] = 20


### Toevoegen stuwen

In [9]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["stuwen"]),
                           "weirs",
                           column_mapping={"kstident": "code",
                                           "kstsoort": "soortstuwcode",
                                           "kstkrubr": "laagstedoorstroombreedte",
                                           "kstmikho": "laagstedoorstroomhoogte",
                                           "kstregel": "soortregelbaarheidcode"})

gdf["afvoercoefficient"] = 1

hydamo.weirs.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.weirs.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=10)
hydamo.weirs.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen gemalen

In [10]:
pumps_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["gemalen"]),
                                 "pumps",
                                 column_mapping={
                                     "KGMIDENT": "code",
                                     "KGMMACAP": "maximalecapaciteit"})

gemalen_gdf = pumps_gdf.copy()
pumps_gdf["codegerelateerdobject"] = pumps_gdf["code"].copy()
pumps_gdf["code"] = [f"PMP_{code}" for code in pumps_gdf["code"]]
hydamo.gemalen.set_data(gemalen_gdf,
                        index_col="code",
                        check_columns=True,
                        check_geotype=True)
hydamo.gemalen.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.gemalen.dropna(axis=0, inplace=True, subset=["branch_offset"])
hydamo.pumps.set_data(pumps_gdf,
                      index_col="code",
                      check_columns=True,
                      check_geotype=True)
hydamo.pumps.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.pumps.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen afsluitmiddelen

In [11]:
gdf = pd.DataFrame({"code": [],
                    "soortafsluitmiddelcode": [],
                    "codegerelateerdobject": []})

hydamo.afsluitmiddel.set_data(gdf, index_col="code")

### Toevoegen sturing

In [12]:
gpg_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["peilgebieden"]),
                                 "sturing",
                                 column_mapping={"GPGIDENT": "code",
                                                 "GPGZMRPL": "streefwaarde"
                                                 })

gpg_gdf["codegerelateerdobject"] = [code.replace("GPG", "") for code in gpg_gdf["code"]]
gpg_gdf = gpg_gdf[gpg_gdf["codegerelateerdobject"].isin(hydamo.gemalen["code"])]
con_gdf = hydamo.gemalen[~hydamo.gemalen["code"].isin(gpg_gdf["codegerelateerdobject"])]
con_gdf = gpd.GeoDataFrame(data={"code": con_gdf["code"].values,
                                 "geometry": con_gdf["geometry"].values})
con_gdf["codegerelateerdobject"] = con_gdf["code"]
con_gdf["streefwaarde"] = -999

gdf = gpd.GeoDataFrame(pd.concat([gpg_gdf, con_gdf], ignore_index=True))

gdf["bovenmarge"] = gdf["streefwaarde"] + 0.05
gdf["ondermarge"] = gdf["streefwaarde"] - 0.05
gdf["doelvariabelecode"] = 1

hydamo.sturing.set_data(gdf, index_col="code")

### Filteren

We maken een subset van branches op basis van waarden uit een gegeven veld. In dit geval worden alle takken waarvoor geldt OBJECTID = 1 opgenomen in de modelschematisatie. Idem voor alle kunstwerken + profielen die naar deze branches zijn gesnapped. Om het resultaat te beoordelen exporteren we alles naar shape-files. We slaan het hydamo object ook op als "pickle", zodat we bovenstaande stappen niet elke keer hoeven te herhalen

In [13]:
hydamo = hydrotools.filter_model(hydamo, attribute_filter={"OBJECTID": 1})
hydrotools.export_shapes(hydamo, path=Path(r"./hydamo_shp/dellen"))
hydrotools.save_model(hydamo, file_name=Path(r"./hydamo_model/dellen.pickle"))


## Converteren naar DFM

### Aanmaken dfm-klasse

In [14]:
dfmmodel = DFlowFMModel()

### Inlezen kunstwerken

In [15]:
dfmmodel.structures.io.weirs_from_hydamo(hydamo.weirs,
                                         yz_profiles=hydamo.crosssections,
                                         parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.culverts_from_hydamo(hydamo.culverts,
                                            hydamo.afsluitmiddel)

dfmmodel.structures.io.bridges_from_hydamo(hydamo.bridges,
                                           yz_profiles=hydamo.crosssections,
                                           parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.orifices_from_hydamo(hydamo.orifices)

dfmmodel.structures.io.pumps_from_hydamo(pompen=hydamo.pumps,
                                         sturing=hydamo.sturing,
                                         gemalen=hydamo.gemalen)

2021-06-08 17:07:11,179 - delft3dfmpy.converters.hydamo_to_dflowfm - hydamo_to_dflowfm - INFO - Currently only simple weirs can be applied. From Hydamo the attributes 'laagstedoorstroomhoogte' and 'kruinbreedte' are used to define the weir dimensions.


### Aanmaken 1d netwerk

In [16]:
dfmmodel.network.set_branches(hydamo.branches)
dfmmodel.network.generate_1dnetwork(one_d_mesh_distance=100.0, seperate_structures=True)

2021-06-08 17:07:11,371 - delft3dfmpy.core.dfm - dfm - WARNING - Some structures are not linked to a branch.
2021-06-08 17:07:11,374 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF-O-00002 at: [0.0, 11.2105, 44.941500000000005, 71.82240223650987], due to the structures at [-0.001, 0.36, 22.061, 67.822, 71.82340223650988].
2021-06-08 17:07:11,375 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF-O-00055 at: [0.0, 167.691, 243.7712895696411], due to the structures at [-0.001, 83.667, 164.841, 170.541, 243.77228956964112].
2021-06-08 17:07:11,378 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF-O-00059 at: [0.0, 525.8205, 985.2394999999999, 1046.7060000000001, 1083.9012346060038], due to the structures at [-0.001, 202.42, 317.606, 519.187, 532.454, 949.228, 1021.251, 1072.161, 1083.9022346060037].
2021-06-08 17:07:11,380 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF-O-00074 at: [0.0, 332.925, 38

### Toevoegen cross-sections

In [17]:
dfmmodel.crosssections.io.from_hydamo(
    dwarsprofielen=hydamo.crosssections,
    parametrised=hydamo.parametrised_profiles,
    branches=hydamo.branches
)

print(f"{len(dfmmodel.crosssections.get_branches_without_crosssection())} branches are still missing a cross section.")
print(f"{len(dfmmodel.crosssections.get_structures_without_crosssection())} structures are still missing a cross section.")

2021-06-08 17:07:11,557 - delft3dfmpy.io.dfmreader - dfmreader - INFO - Before adding the number of branches without cross section is: 172.
2021-06-08 17:07:11,731 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'dwarsprofielen' the number of branches without cross section is: 52.
No parametrised crossections available for branches.
2021-06-08 17:07:11,733 - delft3dfmpy.io.dfmreader - dfmreader - INFO - Before adding the number of structures without cross section is: 7.
2021-06-08 17:07:11,749 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'dwarsprofielen' the number of branches without cross section is: 7.
2021-06-08 17:07:11,788 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'normgeparametriseerd' the number of structures without cross section is: 0.
52 branches are still missing a cross section.
0 structures are still missing a cross section.


### Toevoegen principeprofielen op takken die nog niet over een profiel beschikken

In [18]:
if len(dfmmodel.crosssections.get_branches_without_crosssection()) > 0:
    print("adding trapezium profiles on branches with missing crosssections.")
    dfmmodel = hydrotools.add_trapeziums(dfmmodel, principe_profielen_df)

adding trapezium profiles on branches with missing crosssections.


In [19]:
###Toevoegen randvoorwaarden

In [20]:
rvw_df = pd.read_excel(excelbestanden.joinpath(modelbestanden["randvoorwaarden"]))

for _,row in rvw_df.iterrows():
    branch_id = row["BRANCH_ID"]
    pt = hydamo.branches.loc[branch_id]["geometry"].coords[int(row["COORD"])]
    series = pd.Series(data=[-4.0, -4.0],
                       index=[pd.Timestamp("2000-01-01"),
                              pd.Timestamp("2100-01-01")]
                       )

### Wegschrijven model

In [21]:
dfmmodel.mdu_parameters["refdate"] = 20000101
dfmmodel.mdu_parameters["tstart"] = 0.0 * 3600
dfmmodel.mdu_parameters["tstop"] = 144.0 * 1 * 3600
dfmmodel.mdu_parameters["hisinterval"] = "120. 0. 0."
dfmmodel.mdu_parameters["cflmax"] = 0.7

dimr_path = r"dummypath\run_dimr.bat"
dfmmodel.dimr_path = dimr_path
fm_writer = DFlowFMWriter(dfmmodel, output_dir="dellen", name="dellen")

fm_writer.objects_to_ldb()
fm_writer.write_all()

if Path(r".\dellen\fm\dellen.mdu").exists():
    print("Model is weggeschreven")
else:
    print("Er is geen model geschreven. Waarschijnlijk is iets fout gegaan")

Model is weggeschreven


## Importeren in de D-Hydro suite

Open het model nu in D-Hydro:
1. Open een "Empty Project"
2. In de "Home" ribbon, ga naar "Import" en selecteer "Flow Flexible Mesh Model"
3. Selecteer het bestand "boezemmodel.mdu" in ".\dfm_model\fm\boezemmodel.mdu"
4. Wacht tot het model is geimporteerd.....

Het geimporteerde model is nu zichtbaar in de D-Hydro suite
<img src="png/boezemmodel.png"/>